In [1]:
import pandas as pd
import numpy as np
import random
df = pd.read_excel('GWU Test Case Type Tracker_ABCNY.XLSX')
yy=df[df['Student']=='Yingying Liu']
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']
ofac_list

,uid,name,entity_type
0,36,AEROCARIBBEAN AIRLINES,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-
2,306,BANCO NACIONAL DE CUBA,-0-
3,424,BOUTIQUE LA MAISON,-0-
4,475,CASA DE CUBA,-0-
...,...,...,...
10915,39520,"ADEN, Mohamoud Abdi",individual
10916,39536,"OREKHOV, Yury Yuryevich",individual
10917,39537,NDA NORD-DEUTSCHE INDUSTRIEANLAGENBAU GMBH,-0-
10918,39538,OPUS ENERGY TRADING LLC,-0-


In [146]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original name','Test Case name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original name,Test Case name


In [147]:
zhonglei={}
for index,row in yy.iterrows():
    uid = row['UID']
    theme = row['Theme']
    category = row['Category']
    sub_category = row['Subcategory']
    entity = row['Entity Type']
    if entity == 'Vessel': 
        key = theme+'/'+category+'/'+sub_category
        value = uid+'/'+entity
        zhonglei[key]=value
print(len(zhonglei))
print(zhonglei)

18
{'Name Additions/First Name Part/2 Name Parts added': 'UID-530/Vessel', 'Name Additions/First Name Part/1 Name Part added twice': 'UID-531/Vessel', 'Name Additions/First Name Part/>2 Name Parts added': 'UID-532/Vessel', 'Name Additions/First Name Part/1 Name Part added': 'UID-533/Vessel', 'Name Deletions/First Name Part/>2 name parts removed': 'UID-542/Vessel', 'Name Deletions/First Name Part/1 Name Part removed': 'UID-543/Vessel', 'Name Deletions/First Name Part/2 name parts removed': 'UID-544/Vessel', 'Name Additions/Last Name Part/1 Name Part added twice': 'UID-534/Vessel', 'Name Additions/Last Name Part/2 Name Parts added': 'UID-535/Vessel', 'Name Additions/Last Name Part/1 Name Part added': 'UID-536/Vessel', 'Name Additions/Last Name Part/>2 Name Parts added': 'UID-537/Vessel', 'Name Deletions/Last Name Part/1 Name Part removed': 'UID-545/Vessel', 'Name Deletions/Last Name Part/>2 Name Parts removed': 'UID-546/Vessel', 'Name Deletions/Last Name Part/2 Name Parts removed': 'UID-

In [148]:
from random_words import RandomWords
def is_proper(sentence,category,sub_category):
    word = sentence.split()
    if sub_category.lower() == '1 name part removed':
        return len(word)>= 2
    elif sub_category.lower() == '>2 name parts removed':
        return len(word)>=4
    elif sub_category.lower() == '2 name parts removed':
        return len(word)>= 3
    elif category.lower() == 'name part in the middle':
        return len(word)>= 3
    else:
        return True

def name_part(sentence,pos,mode,num):
    word = sentence.upper().split()
    rw = RandomWords()
    st = 0
    ed = len(word)-1
    if pos == 'first':
        st = 0
        ed = len(word)-1
    elif pos == 'last':
        st = 1
        ed = len(word)
    else:
        st = 1
        ed = len(word)-1

    if mode == 'add':
        for i in range(num):
            idx = random.randint(st,ed)
            word.insert(idx,rw.random_words(count=1)[0])
    elif mode == 'add_twice':
        name = rw.random_words(count=1)[0].upper()
        
        idx = random.randint(st,ed)
        word.insert(idx,name)
        word.insert(idx,name)
    elif mode == 'remove':
        idx_list = random.sample(range(st,ed),num)
        word = [word[i] for i in range(len(word)) if i not in idx_list]
    return ' '.join(word)


In [149]:
def get_sample(ofac_list,theme,category,sub_category,entity):
    ofac_list_sample = ofac_list[(ofac_list.entity_type == 'vessel')].sample(n=500)
    res = []
    for index,row in ofac_list_sample.iterrows():
        name = row['name']
        uid = row['uid']
        if entity == 'vessel':
            if is_proper(name,category,sub_category):
                res.append(uid)
        if len(res)==10:
            break
    print("sample ",category,' ',sub_category ," ",len(res))
    return res
    
# get_sample(ofac_list,'OFAC','First name part','1 name part removed','vessel')

In [150]:
def func(key,value):
    uid,entity = value.split('/')
    theme,category,sub_category = key.split('/')
    # print(theme,'/',category,'/',sub_category,'/',entity)
    # find test case
    sample_uid = get_sample(ofac_list,theme,category,sub_category,entity.lower())
    # print(sample_uid,'       ',sub_category)
    ofac_list_sampled = ofac_list[ofac_list['uid'].isin(sample_uid)]
    for index,row in ofac_list_sampled.iterrows():
        ofac_uid = row['uid']
        ofac_name = row['name']
        # print(category.lower())
        final_test_name=None
        if category.lower() == 'first name part':
            if sub_category.lower() == '2 name parts added':
                final_test_name = name_part(ofac_name,'first','add',2)
            elif sub_category.lower() == '1 name part added twice':
                final_test_name = name_part(ofac_name,'first','add_twice',1)
            elif sub_category.lower() == '>2 name parts added':
                final_test_name = name_part(ofac_name,'first','add',3)
            elif sub_category.lower() == '1 name part added':
                final_test_name = name_part(ofac_name,'first','add',1)
            elif sub_category.lower() == '>2 name parts removed':
                final_test_name = name_part(ofac_name,'first','remove',3)
            elif sub_category.lower() == '1 name part removed':
                final_test_name = name_part(ofac_name,'first','remove',1)
            elif sub_category.lower() == '2 name parts removed':
                final_test_name = name_part(ofac_name,'first','remove',2)
        elif category.lower() == 'last name part':
            if sub_category.lower() == '2 name parts added':
                final_test_name = name_part(ofac_name,'last','add',2)
            elif sub_category.lower() == '1 name part added twice':
                final_test_name = name_part(ofac_name,'last','add_twice',1)
            elif sub_category.lower() == '>2 name parts added':
                final_test_name = name_part(ofac_name,'last','add',3)
            elif sub_category.lower() == '1 name part added':
                final_test_name = name_part(ofac_name,'last','add',1)
            elif sub_category.lower() == '>2 name parts removed':
                final_test_name = name_part(ofac_name,'last','remove',3)
            elif sub_category.lower() == '1 name part removed':
                final_test_name = name_part(ofac_name,'last','remove',1)
            elif sub_category.lower() == '2 name parts removed':
                final_test_name = name_part(ofac_name,'last','remove',2)
        else:
            if sub_category.lower() == '1 name part added':
                final_test_name = name_part(ofac_name,'middle','add',1)
            elif sub_category.lower() == '2 name parts added':
                final_test_name = name_part(ofac_name,'middle','add',2)
            elif sub_category.lower() == '1 name part added twice':
                final_test_name = name_part(ofac_name,'middle','add_twice',1)
            else:
                final_test_name = name_part(ofac_name,'middle','add',3)
        final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name]


In [151]:
for key,value in zhonglei.items():
    func(key,value)

sample  First Name Part   2 Name Parts added   10
sample  First Name Part   1 Name Part added twice   10
sample  First Name Part   >2 Name Parts added   10
sample  First Name Part   1 Name Part added   10
sample  First Name Part   >2 name parts removed   10
sample  First Name Part   1 Name Part removed   10
sample  First Name Part   2 name parts removed   10
sample  Last Name Part   1 Name Part added twice   10
sample  Last Name Part   2 Name Parts added   10
sample  Last Name Part   1 Name Part added   10
sample  Last Name Part   >2 Name Parts added   10
sample  Last Name Part   1 Name Part removed   10
sample  Last Name Part   >2 Name Parts removed   10
sample  Last Name Part   2 Name Parts removed   10
sample  Name Part in the Middle   1 Name Part added   10
sample  Name Part in the Middle   >2 name parts added   10
sample  Name Part in the Middle   1 name part added twice   10
sample  Name Part in the Middle   2 name parts added   10


In [152]:
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original name,Test Case name
0,UID-530,Name Additions,First Name Part,2 Name Parts added,Vessel,UID-530 - 2755,15042,DIAMOND II,guess DIAMOND sewer II
1,UID-530,Name Additions,First Name Part,2 Name Parts added,Vessel,UID-530 - 2761,15048,DOWNY,societies scratchpads DOWNY
2,UID-530,Name Additions,First Name Part,2 Name Parts added,Vessel,UID-530 - 2786,15075,BELEMA LIGHT CRUDE,splashes BELEMA implantation LIGHT CRUDE
3,UID-530,Name Additions,First Name Part,2 Name Parts added,Vessel,UID-530 - 5664,23517,KUM UN SAN,season destroyer KUM UN SAN
4,UID-530,Name Additions,First Name Part,2 Name Parts added,Vessel,UID-530 - 5792,23738,JI SONG 6,JI blocks SONG respects 6
...,...,...,...,...,...,...,...,...,...
175,UID-541,Name Additions,Name Part in the Middle,2 name parts added,Vessel,UID-541 - 5476,23163,KUM UN SAN 3,KUM alkalinity periods UN SAN 3
176,UID-541,Name Additions,Name Part in the Middle,2 name parts added,Vessel,UID-541 - 5479,23166,RUNG RA DO,RUNG fifties berries RA DO
177,UID-541,Name Additions,Name Part in the Middle,2 name parts added,Vessel,UID-541 - 5481,23168,RUNG RA 1,RUNG RA proportions belief 1
178,UID-541,Name Additions,Name Part in the Middle,2 name parts added,Vessel,UID-541 - 5494,23185,KU BONG RYONG,KU comma BONG explosions RYONG
